In [1]:
import dlib
import cv2
import numpy as np
import imutils

In [2]:
from scipy.spatial import distance as dist
from imutils import face_utils
from threading import Thread
from gtts import gTTS
from playsound import playsound

In [3]:
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html

import glob

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6 * 7, 3), np.float32)
objp[:, :2] = np.mgrid[0: 7, 0: 6].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane.

images = glob.glob('data/images/*.jpg')
print(images)

for image in images:
    img = cv2.imread(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, (7, 6), None)

    if ret:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

        img = cv2.drawChessboardCorners(img, (7, 6), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        print(mtx)
        print(dist)

cv2.destroyAllWindows()

['data/images/left04.jpg', 'data/images/left06.jpg', 'data/images/left09.jpg', 'data/images/1_joey-friends.jpg', 'data/images/left07.jpg', 'data/images/left05.jpg', 'data/images/left01.jpg', 'data/images/left03.jpg', 'data/images/left14.jpg', 'data/images/joye.jpg', 'data/images/left11.jpg', 'data/images/left12.jpg', 'data/images/left.jpg', 'data/images/left08.jpg', 'data/images/left13.jpg', 'data/images/left02.jpg']
[[515.51273861   0.         341.23697124]
 [  0.         515.48750138 234.27066711]
 [  0.           0.           1.        ]]
[[-2.04120057e-01 -1.03446509e+00  2.31949959e-03 -2.67680902e-03
   5.40565638e+00]]
[[568.84318872   0.         305.45921311]
 [  0.         576.68385278 235.16080587]
 [  0.           0.           1.        ]]
[[-0.3417715   0.21139459  0.00077113  0.01687714 -0.20818868]]
[[564.3049453    0.         313.66492301]
 [  0.         570.56795432 234.49413053]
 [  0.           0.           1.        ]]
[[-0.34029736  0.21984852  0.00085473  0.0145984

In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector1 = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [5]:
def ref3DModel():
    modelPoints = [[0.0,0.0,0.0],
                   [0.0,-330.0,-65.0],
                   [-255.0,170.0,-135.0],
                   [225.0,170.0,-135.0],
                   [-150.0,-150.0,-125.0],
                   [150.0,-150.0,-125.0]]
    return np.array(modelPoints,dtype=np.float64)

In [6]:
def ref2DImagePoints(shape):
    imagePoints = [[shape.part(30).x,shape.part(30).y],
                   [shape.part(8).x,shape.part(8).y],
                   [shape.part(36).x,shape.part(36).y],
                   [shape.part(45).x,shape.part(45).y],
                   [shape.part(48).x,shape.part(48).y],
                   [shape.part(54).x,shape.part(54).y]]
    return np.array(imagePoints,dtype=np.float64)

In [7]:
# def CameraMatrix(fl,center):
#     cameraMatrix = [[fl,1,center[0]],
#                     [0,fl,centre[1]],
#                     [0,0,1]]
#     return np.array(cameraMatrix,dtype=np.float)

In [8]:
def drawPolyline(img,shapes,start,end,isClosed=False):
    points = []
    for i in range(start,end+1):
        point = [shapes.part(i).x,shapes.part(i).y]
        points.append(point)
    points = np.array(points,dtype=np.float32)
    cv2.polylines(img,np.int32([points]),isClosed,(255,80,0),thickness=1,lineType=cv2.LINE_8)

In [9]:
def draw(img,shapes):
    drawPolyline(img, shapes, 0, 16)
    drawPolyline(img, shapes, 17, 21)
    drawPolyline(img, shapes, 22, 26)
    drawPolyline(img, shapes, 27, 30)
    drawPolyline(img, shapes, 30, 35, True)
    drawPolyline(img, shapes, 36, 41, True)
    drawPolyline(img, shapes, 42, 47, True)
    drawPolyline(img, shapes, 48, 59, True)
    drawPolyline(img, shapes, 60, 67, True)

In [10]:
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 30
YAWN_THRESH = 20
alarm_status = False
alarm_status2 = False
saying = False
COUNTER = 0

In [11]:
def alarm(m):
    while alarm_status:
        print("Call")
        myText = m
        language = 'en'
        output = gTTS(text=myText,lang=language,slow=False)
        output.save("output.mp3")
        playsound("output.mp3")
    
    if alarm_status2:
        print("Call")
        saying = True
        myText = m
        language = 'en'
        output = gTTS(text=myText,lang=language,slow=False)
        output.save("output.mp3")
        playsound("output.mp3")
        saying = False

In [12]:
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1]-eye[5])
    B = np.linalg.norm(eye[2]-eye[4])
    C = np.linalg.norm(eye[0]-eye[3])
    
    ear = (A+B)/(2.0*C)
    
    return ear

In [13]:
def final_ear(shape):
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    
    ear = (leftEAR+rightEAR)/2.0
    return (ear,leftEye,rightEye)

In [14]:
def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))
    
    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))
    
    top_mean = np.mean(top_lip,axis=0)
    low_mean = np.mean(low_lip,axis=0)
    
    distance = abs(top_mean[1]-low_mean[1])
    return distance

In [15]:
cap = cv2.VideoCapture(0)

while True:
    GAZE = "Face Not Found"
    ret,img = cap.read()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rects = detector1.detectMultiScale(gray, scaleFactor=1.1,minNeighbors=5, minSize=(30, 30),flags=cv2.CASCADE_SCALE_IMAGE)

    #for rect in rects:
    for (x, y, w, h) in rects:
        rect = dlib.rectangle(int(x), int(y), int(x + w),int(y + h))
        
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        eye = final_ear(shape)
        ear = eye[0]
        leftEye = eye [1]
        rightEye = eye[2]
        
#         leftEyeHull = cv2.convexHull(leftEye)
#         rightEyeHull = cv2.convexHull(rightEye)
#         cv2.drawContours(img, [leftEyeHull], -1, (0, 0, 255), 1)
#         cv2.drawContours(, [rightEyeHull], -1, (0, 0, 255), 1)

        if ear < EYE_AR_THRESH:
            COUNTER += 1

            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                if alarm_status == False:
                    alarm_status = True
                    t = Thread(target=alarm, args=('wake up sid',))
                    t.deamon = True
                    t.start()

                cv2.putText(img, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        else:
            COUNTER = 0
            alarm_status = False
     
    faces = detector(cv2.cvtColor(img,cv2.COLOR_BGR2RGB),0)
    face3Dmodel = ref3DModel()
    
    for face in faces:
        shape = predictor(cv2.cvtColor(img,cv2.COLOR_BGR2RGB),face)
        
        draw(img,shape)
        
        shape1 = face_utils.shape_to_np(shape)

        distance = lip_distance(shape1)

        refImgPts = ref2DImagePoints(shape)
        
        height,width,channels = img.shape
#         focalLength = args.focal * width
#         cameraMatrix = cameraMatrix(focalLength,(height/2,width/2))
        
        mdists = np.zeros((4,1),dtype=np.float64)
        
        success,rotationVector,translationVector = cv2.solvePnP(face3Dmodel,refImgPts,mtx,mdists)
        
        noseEndPoints3D = np.array([[0,0,1000.0]],dtype=np.float64)
        noseEndPoint2D,jacobian = cv2.projectPoints(noseEndPoints3D, rotationVector, translationVector, mtx, mdists)
        
        rmat, jac = cv2.Rodrigues(rotationVector)
        angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
        
        if angles[1] < -15:
            GAZE = "Looking: Left"
        elif angles[1] > 15:
            GAZE = "Looking: Right"
        else:
            GAZE = "Forward"
            
#         if ear < EYE_AR_THRESH:
#             COUNTER += 1

#             if COUNTER >= EYE_AR_CONSEC_FRAMES:
#                 if alarm_status == False:
#                     alarm_status = True
#                     t = Thread(target=alarm, args=('wake up sir',))
#                     t.deamon = True
#                     t.start()

#                 cv2.putText(img, "DROWSINESS ALERT!", (10, 30),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

#         else:
#             COUNTER = 0
#             alarm_status = False

        if (distance > YAWN_THRESH):
                cv2.putText(img, "Yawn Alert", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            alarm_status2 = False

            
    cv2.putText(img,GAZE,(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,80),2)
    cv2.imshow("Head Pose",img)
    
    key = cv2.waitKey(10) & 0xFF
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

Call
Call
